In [1]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("grouplens/movielens-20m-dataset")

print("Path to dataset files:", path)

Path to dataset files: /Users/riyaazkhan/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1


In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import sklearn.tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from scipy.sparse import coo_matrix


/Users/riyaazkhan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/riyaazkhan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [30]:
genome = pd.read_csv(path + '/genome_scores.csv')
genometags = pd.read_csv(path + '/genome_tags.csv')
links = pd.read_csv(path + '/link.csv')
movies = pd.read_csv(path + '/movie.csv')
ratings = pd.read_csv(path + '/rating.csv')
tags = pd.read_csv(path + '/tag.csv')


In [31]:
def find_duplicates(df, column_name):
    # Find all duplicate values
    duplicates = df[column_name][df[column_name].duplicated()].unique()
    
    # Convert to a list
    return list(duplicates)
#movies = (movies.drop_duplicates(subset='title',keep='first'))

In [32]:
duplicates = find_duplicates(movies,'title')

In [33]:
duplicates_IDs = []
for i in range(len(duplicates)):
    x = duplicates[i]
    ID = (movies[movies['title'] == x]['movieId'].squeeze())
    for item in ID:
        duplicates_IDs.append(item)
for i in range(len(duplicates_IDs)):
    ratings = ratings[ratings['movieId'] != duplicates_IDs[i]]
    tags = tags[tags['movieId'] != duplicates_IDs[i]]
    
for i in range(len(duplicates)):
    movies = movies[movies['title'] != duplicates[i]]
print(len(movies))
print(duplicates_IDs)

27246
[838, 26958, 1788, 26982, 34048, 64997, 3598, 65665, 47254, 67459, 48682, 80330, 66140, 85070, 97773, 101212, 102190, 114130, 588, 114240, 104035, 115777, 113459, 121586, 104155, 122940, 42015, 128862, 111519, 128991, 93279, 130062]


In [34]:
reducedratings = ratings.head(4000000)
reducedtags = tags
reducedratingstitles = []
reducedratings

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
4011714,27299,520,4.0,2000-11-08 04:12:15
4011715,27299,527,4.0,2000-11-08 03:47:16
4011716,27299,537,1.0,2000-11-08 04:18:00
4011717,27299,541,4.0,2002-03-23 19:24:28


In [35]:
#film_user_ratings = reducedratings.pivot(index='movieId',columns='userId',values = 'rating').fillna(0)
# Filter for movies and users with sufficient ratings
min_ratings_per_movie = 10
min_ratings_per_user = 5
valid_movies = film_user_ratings.sum(axis=1) >= min_ratings_per_movie
valid_users = film_user_ratings.sum(axis=0) >= min_ratings_per_user
film_user_ratings = film_user_ratings.loc[valid_movies, valid_users]

film_user_ratings = film_user_ratings.sub(film_user_ratings.mean(axis=1), axis=0)

In [36]:
film_user_ratings = reducedratings.pivot(index='movieId',columns='userId',values = 'rating').fillna(0)


In [37]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(film_user_ratings)
similarity_df = pd.DataFrame(similarity_matrix)  

In [38]:
def remove_duplicate_columns(df):
    df.columns = df.columns.astype(str)

    # Find the duplicate column names
    seen = set()
    columns_to_keep = []
    for col in df.columns:
        if col not in seen:
            columns_to_keep.append(col)
            seen.add(col)

    # Retain only the columns that are not duplicates
    return df.loc[:, columns_to_keep]

In [39]:
def remove_duplicate_indices(df):

    df.index = df.index.map(str)
    
    # Identify duplicate indices and keep the first occurrence
    return df[~df.index.duplicated(keep='first')]

In [40]:

y = similarity_df.index 

x=[]
for item in y:
    x.append(movies.iloc[item]['title'])
    #x.append(movies[movies['movieId'] == (item+1)]['title'].squeeze())
similarity_df.index = x
similarity_df.columns = x

#similarity_df = remove_duplicate_columns(similarity_df)
#similarity_df = remove_duplicate_indices(similarity_df)

movies_reduced = movies[movies['title'].isin(similarity_df.index)]# Filter movies where title is in the similarity_df.index

print(len(movies_reduced['title']))
print(len(similarity_df.index))


19354
19354


KeyboardInterrupt: 

In [73]:
genres_split

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19366,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
19367,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
19368,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0
19369,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [41]:
# Incorporating Genre Split.
# This one-hot encodes the genres.
genres_split = movies_reduced['genres'].str.get_dummies(sep='|')
movie_genres = pd.concat([movies, genres_split], axis=1)


In [42]:
#index2 =[]
#for i in range(len(movies_reduced)):
#    index2.append((movies[movies['movieId'] == movies_reduced.index[i]]['title']).squeeze()) # Used to change the columns and rows of simalrity_df
from sklearn.metrics.pairwise import cosine_similarity
genre_similarity = cosine_similarity(genres_split)
genre_similarity_df = pd.DataFrame(genre_similarity, index=similarity_df.index,columns=similarity_df.index)


In [106]:
#Cosine Similarity algorithm self-made
genres_split_T = genres_split.T
A = genres_split_T['Toy Story (1995)']
B = genres_split_T['Jumanji (1995)']
(A @ B)/ (np.linalg.norm(A) * np.linalg.norm(B))

0.7745966692414834

In [115]:
genres_similarity_cosine = pd.DataFrame(0,index=x,columns=x)

In [114]:
genres_similarity_cosine = pd.DataFrame(0,index=x,columns=x,dtype='float64')
for item1 in genres_split_T:
    A = genres_split_T[item1]
    for item2 in genres_split_T:
        B = genres_split_T[item2]
        score = (A @ B)/ (np.linalg.norm(A) * np.linalg.norm(B))
        genres_similarity_cosine.loc[item1,item2] = score
genre_similarity_df = genres_similarity_cosine

KeyboardInterrupt: 

Index(['(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
       'War', 'Western'],
      dtype='object')

In [98]:
#Jaccard similarity
genres_split_names = genres_split
genres_split_names.index = x
genres_split_names
genre_similarity_jaccard =pd.DataFrame(0,index=x,columns=x,dtype='float64')
genre_similarity_jaccard
for item1 in genre_similarity_jaccard:
    for item2 in genre_similarity_jaccard:
        similarity_score = 0
        unsimilar_score = 0
        for column in genres_split_names.columns:
            if genres_split_names[column][item1] == 1 and genres_split_names[column][item2] == 1:
                similarity_score += 1
            elif genres_split_names[column][item1] != genres_split_names[column][item2]:
                unsimilar_score += 1
        if similarity_score + unsimilar_score == 0:
            total_score = 0
        else:
            total_score = score/(len(genres_split_names.columns))
        genre_similarity_jaccard.loc[item1,item2] = float(total_score)
genre_similarity_df = genre_similarity_jaccard

KeyboardInterrupt: 

In [63]:
genres_split

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19366,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
19367,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
19368,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0
19369,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [57]:
(similarity_df)['Toy Story 2 (1999)']

Toy Story (1995)                      0.033032
Jumanji (1995)                        0.018333
Grumpier Old Men (1995)               0.017135
Waiting to Exhale (1995)              0.015875
Father of the Bride Part II (1995)    0.022529
                                        ...   
Human Trafficking (2005)             -0.000465
Kidulthood (2006)                    -0.000465
Like Mike 2: Streetball (2006)       -0.000458
Lady in Cement (1968)                -0.000489
Street Trash (1987)                  -0.000534
Name: Toy Story 2 (1999), Length: 11649, dtype: float64

In [48]:
rating_weight = 0.5
genre_weighting = 0.5
combined_similarity_table = (
    (rating_weight * similarity_df) +
    (genre_weighting * genre_similarity_df))

In [121]:
combined_similarity_table.head(4952)

,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Secret Reunion (Ui-hyeong-je) (2010),War of the Dead - Stone's War (2011),Super Rich: The Greed Game (2008),"One Percent, The (2006)",Primal (2010),Dreams That Money Can Buy (1947),Profound Desires of the Gods (Kamigami no fukaki yokubo) (1968),"See You in Hell, My Darling (Tha se do stin Kolasi, agapi mou) (1999)","Rickshaw Man, The (Muhomatsu no issho) (1958)",Identification Marks: None (Rysopis) (1965)
Toy Story (1995),1.000000,0.592567,0.311042,0.190005,0.373108,0.194529,0.311487,0.368312,0.083298,0.322394,...,0.000000,0.091287,0.0,0.005035,0.003776,0.223607,0.000000,0.000000,0.158114,0.005664
Jumanji (1995),0.592567,1.000000,0.119674,0.084012,0.135191,0.144561,0.118686,0.502979,0.072602,0.377805,...,0.000000,0.117851,0.0,0.000000,0.000000,0.293174,0.004498,0.000000,0.000000,0.006748
Grumpier Old Men (1995),0.311042,0.119674,1.000000,0.486349,0.580701,0.139197,0.697120,0.069939,0.126942,0.101601,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.012089
Waiting to Exhale (1995),0.190005,0.084012,0.486349,1.000000,0.376954,0.063319,0.499918,0.066493,0.061944,0.067741,...,0.204124,0.000000,0.0,0.000000,0.000000,0.000000,0.288675,0.000000,0.408248,0.288675
Father of the Bride Part II (1995),0.373108,0.135191,0.580701,0.376954,1.000000,0.122170,0.554344,0.080350,0.113429,0.102105,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.353553,0.011117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kung Pow: Enter the Fist (2002),0.231318,0.078572,0.298058,0.227670,0.394442,0.271587,0.294721,0.027871,0.377841,0.270327,...,0.000000,0.144338,0.0,0.022523,0.000000,0.000000,0.000000,0.000000,0.250000,0.022523
Snow Dogs (2002),0.442157,0.379533,0.234233,0.189773,0.326707,0.024536,0.249922,0.443240,0.007363,0.194920,...,0.000000,0.117851,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.204124,0.000000
48 Hrs. (1982),0.116465,0.004486,0.176777,0.299566,0.250000,0.295257,0.193209,0.000000,0.260341,0.150075,...,0.176777,0.102062,0.0,0.000000,0.000000,0.000000,0.250000,0.144338,0.353553,0.250000
"Farewell, The (Abschied - Brechts letzter Sommer) (2000)",0.013391,0.012271,0.005400,0.291934,0.002079,0.014428,0.005630,0.004394,0.004950,0.011445,...,0.353553,0.000000,0.0,0.000000,0.000000,0.000000,0.500000,0.000000,0.353553,0.500000


In [127]:
def find_similar_films(FilmName,n=10):
    similar_movies = combined_similarity_table[FilmName].sort_values(ascending=False)
    SimilarFilmId = (similar_movies.index)[1:n+1]
    return similar_movies[1:n+1]
    
user_input = input('What Film have you seen that you would like reccomendations for?')
if user_input in (movies["title"].values):
    print('ok')
    films = find_similar_films(user_input,10)
    print(films)
    
        
else:
    print('Movie not in database')


What Film have you seen that you would like reccomendations for?Atlantis: Milo's Return (2003)
ok
Pokemon: The Movie 2000 (2000)                                                                                                                       0.908248
Niko & the Way to the Stars (a.k.a. The Flight Before Christmas) (Niko - Lentäjän poika) (2008)                                                      0.865148
Hop (2011)                                                                                                                                           0.853553
Hobbit, The (1977)                                                                                                                                   0.853553
Versus (2000)                                                                                                                                        0.806186
Scooby-Doo! Curse of the Lake Monster (2010)                                                                    